# Monte Carlo perturbations of DEM and basin delineation

This notebook documents how we generate and analyse an **ensemble of hydrological basin maps**
by perturbing the input DEM according to its elevation uncertainty.

The goals are to:

1. Sample many realisations of the DEM using its variance/standard‑deviation raster.
2. For each realisation, run the full GRASS hydrological workflow to obtain a basin map.
3. Derive **pixelwise** and **basinwise** stability metrics that tell us where basin boundaries
   are robust and where they are uncertain.


## 1. Configure DEM, uncertainty and GRASS environment

This cell:

- Defines paths to the DEM (`PRODEM19_dem.tif`) and its variance / uncertainty raster.
- Sets the number of Monte Carlo realisations `N_MC` and the stream extraction threshold.
- Sets up a GRASS location (via QGIS) and imports the DEM and ice mask.
- For each Monte Carlo run:

  * Draws a random perturbation field consistent with the DEM uncertainty.
  * Adds it to the DEM to create a perturbed DEM realisation.
  * Runs the hydrological workflow (flow routing, stream extraction, `r.stream.basins`).
  * Writes the resulting basin map to `basins_mc_XXX.tif` in the output directory.


In [3]:
import os
import sys
from pathlib import Path

# =============================================================================
# USER SETTINGS
# =============================================================================
DEM      = r"E:\Rasmus\DTU\Cryo\4DGreenland\DEM\PRODEM19_dem.tif"
VAR      = r"E:\Rasmus\DTU\Cryo\4DGreenland\DEM\prodem19_var.tif"  # variance (σ² or σ, see note below)
ice_mask = r"E:\Rasmus\DTU\Cryo\4DGreenland\Ice_mask\02-PROMICE-2022-IceMask-polygon.gpkg"
OUT      = r"E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC2"

QGIS_PREFIX      = r"C:\Program Files\QGIS 3.40.11"
STREAM_THRESHOLD = 500   # stream extraction threshold
N_MC             = 2    # number of Monte Carlo realisations
CORR_PIX         = 5     # correlation length in pixels for Gaussian kernel
REF_RUN          = 1     # reference realisation for basin stability
STABLE_SUM_MAP   = "basin_match_sum"  # internal GRASS raster for stability count
CERT_THRESH      = 0.9   # certainty threshold for "core" basins

Path(OUT).mkdir(parents=True, exist_ok=True)

# =============================================================================
# GRASS ENVIRONMENT INIT
# =============================================================================
GISBASE = fr"{QGIS_PREFIX}\apps\grass\grass84"
os.environ.update({
    "GISBASE": GISBASE,
    "GRASS_PYTHON": sys.executable,
    "PROJ_LIB": fr"{QGIS_PREFIX}\share\proj",
    "GDAL_DATA": fr"{QGIS_PREFIX}\share\gdal",
})

os.environ["PATH"] = os.pathsep.join([
    fr"{GISBASE}\bin",
    fr"{GISBASE}\extrabin",
    fr"{QGIS_PREFIX}\bin",
    fr"{QGIS_PREFIX}\apps\Qt5\bin",
    os.environ["PATH"],
])

sys.path.insert(0, fr"{GISBASE}\etc\python")

import grass.script as gs
import grass.script.setup as gsetup
from grass.script.core import find_program, CalledModuleError

# =============================================================================
# HELPERS
# =============================================================================
def start_grass_from_raster(raster_path, location="dem_loc", mapset="PERMANENT"):
    """Start GRASS in ~/Documents/grassdata based on a DEM."""
    gisdbase = Path.home() / "Documents" / "grassdata"
    gisdbase.mkdir(parents=True, exist_ok=True)

    loc_path = gisdbase / location
    mapset_path = loc_path / mapset

    if not loc_path.exists():
        print(f"📁 Creating new GRASS location: {loc_path}")
        gs.core.create_location(
            dbase=str(gisdbase),
            location=location,
            filename=raster_path,
            overwrite=True
        )
    else:
        print(f"✅ Using existing GRASS location: {loc_path}")

    gsetup.init(str(mapset_path))
    print(f"🌿 GRASS session initialized in:\n   {mapset_path}\n")
    print(gs.read_command("g.gisenv"))

    return str(gisdbase), location, mapset


def import_dem_native(input_path, out_name="dem"):
    """Import or clone DEM to a native GRASS raster."""
    raster = input_path.replace("\\", "/")
    try:
        gs.run_command(
            "r.in.gdal",
            input=raster,
            output=out_name,
            flags="o",
            overwrite=True
        )
        print(f"✓ r.in.gdal → {out_name}")
    except Exception:
        gs.run_command(
            "r.external",
            input=raster,
            output=f"{out_name}_ext",
            flags="o",
            overwrite=True
        )
        gs.run_command("g.region", raster=f"{out_name}_ext")
        gs.mapcalc(f"{out_name} = {out_name}_ext * 1.0", overwrite=True)
        print(f"✓ r.external + clone → {out_name}")
    gs.run_command("g.region", raster=out_name)


def safe(expr: str):
    """Convenience wrapper for r.mapcalc with error reporting."""
    try:
        gs.run_command("r.mapcalc", expression=expr, overwrite=True)
    except CalledModuleError as e:
        raise RuntimeError(f"Mapcalc failed: {expr}\n{e}")


def ensure_grass_addon(module_name: str):
    """Install a GRASS addon if missing."""
    if find_program(module_name) is None:
        gs.run_command(
            "g.extension",
            extension=module_name,
            operation="add",
            flags="f"
        )
        if find_program(module_name) is None:
            raise RuntimeError(f"Failed to install GRASS addon: {module_name}")
    print(f"✓ Addon available: {module_name}")


# =============================================================================
# START GRASS AND IMPORT DEM + VARIANCE
# =============================================================================
GISDBASE, LOCATION, MAPSET = start_grass_from_raster(DEM)
import_dem_native(DEM, out_name="dem")
import_dem_native(VAR, out_name="dem_var")
print(gs.read_command("g.version"))

# =============================================================================
# GRASS ADDON HANDLING
# =============================================================================
APPDATA = os.environ.get("APPDATA", str(Path.home()))
addon_base = Path(APPDATA) / "GRASS8" / "addons"
os.environ["GRASS_ADDON_BASE"] = str(addon_base)

addon_bin = addon_base / "bin"
addon_scripts = addon_base / "scripts"

os.environ["PATH"] = os.pathsep.join([
    str(addon_bin),
    str(addon_scripts),
    os.environ["PATH"]
])
os.environ["GRASS_ADDON_PATH"] = os.pathsep.join([str(addon_bin), str(addon_scripts)])

ensure_grass_addon("r.stream.extract")
ensure_grass_addon("r.stream.basins")

print(gs.read_command("g.gisenv"))
print("ADDON_BASE:", os.environ["GRASS_ADDON_BASE"])
print("PATH contains addon_bin?", str(addon_bin) in os.environ["PATH"])
print("PATH contains addon_scripts?", str(addon_scripts) in os.environ["PATH"])

# make sure no mask is active initially
try:
    gs.run_command("r.mask", flags="r")
except Exception:
    pass

# =============================================================================
# FILL ONLY INTERNAL HOLES IN DEM (ONCE, WITHOUT MASK)
# =============================================================================
gs.run_command("g.region", raster="dem")
gs.run_command(
    "r.fillnulls",
    input="dem",
    output="dem_filled",
    method="bilinear",
    overwrite=True
)
print("✓ Filled internal DEM holes → dem_filled")

# =============================================================================
# IMPORT ICE MASK ONCE AND RASTERIZE
# =============================================================================
gs.run_command(
    "v.import",
    input=ice_mask.replace("\\", "/"),
    output="ice_mask_vec",
    overwrite=True
)

gs.run_command("g.region", raster="dem_filled")
gs.run_command(
    "v.to.rast",
    input="ice_mask_vec",
    output="ice_mask_rast",
    use="val",
    value=1,
    overwrite=True
)
print("✓ Ice mask imported and rasterized → ice_mask_rast")

# =============================================================================
# MONTE CARLO HELPER FUNCTIONS
# =============================================================================
def make_perturbed_dem(run_idx,
                       base_dem="dem_filled",
                       var_map="dem_var",
                       corr_pix=CORR_PIX):
    """
    Create a spatially correlated random perturbation and add it to the DEM.
    Uses a Gaussian kernel with correlation length specified in pixels.

    Assumes var_map contains variance (σ²). If it contains σ instead, replace
    sqrt(var_map) with var_map in the perturbation expression.
    """
    # ensure no mask is active while generating noise / perturbation
    try:
        gs.run_command("r.mask", flags="r")
    except Exception:
        pass

    # region aligned to DEM
    gs.run_command("g.region", raster=base_dem)

    # 1) Gaussian white noise N(0,1)
    noise_raw = "noise_raw"
    gs.run_command(
        "r.surf.gauss",
        output=noise_raw,
        mean=0.0,
        sigma=1.0,
        overwrite=True
    )

    # 2) Impose spatial correlation with Gaussian kernel (corr_pix in cells)
    reg = gs.parse_command("g.region", flags="g")
    cellsize = float(reg["ewres"])  # assume square-ish pixels

    radius1 = corr_pix * cellsize          # in map units
    radius2 = 2 * corr_pix * cellsize      # in map units

    noise_corr = "noise_corr"
    gs.run_command(
        "r.resamp.filter",
        input=noise_raw,
        output=noise_corr,
        filter="gauss,box",
        radius=f"{radius1},{radius2}",
        overwrite=True
    )

    # 3) Build perturbation and perturbed DEM
    out_dem  = f"dem_mc_{run_idx:03d}"
    pert_map = f"pert_mc_{run_idx:03d}"

    # If VAR is actually σ (std dev), change sqrt({var_map}) → {var_map}
    safe(
        f"{pert_map} = if(isnull({base_dem}) || isnull({var_map}), "
        f"null(), float(noise_corr * sqrt({var_map})))"
    )

    safe(
        f"{out_dem} = if(isnull({base_dem}), null(), {base_dem} + {pert_map})"
    )

    print(f"✓ Created perturbation: {pert_map}")
    print(f"✓ Created perturbed DEM: {out_dem}")

    return out_dem


def run_hydro_for_dem(dem_name, run_idx):
    """
    Run neighbors → fillnulls → watershed → streams → basins
    for a given DEM and export rasters with run-specific filenames.
    Also updates the basin stability accumulator relative to REF_RUN.
    """
    # Region to DEM
    gs.run_command("g.region", raster=dem_name, flags="p")

    # Apply ice mask (now DEM is clean; we don't fill based on mask)
    gs.run_command("r.mask", raster="ice_mask_rast", overwrite=True)
    print("✓ Mask set from ice_mask_rast")

    # Smooth and fill any residual NULLs (inside mask)
    gs.run_command(
        "r.neighbors",
        input=dem_name,
        output="dem_smoothed_raw",
        method="average",
        size=3,
        memory=300,
        overwrite=True
    )

    gs.run_command(
        "r.fillnulls",
        input="dem_smoothed_raw",
        output="dem_smoothed",
        method="bilinear",
        overwrite=True
    )

    # Hydrology module outputs
    accum    = f"accum_{run_idx:03d}"
    flow_dir = f"flow_dir_{run_idx:03d}"
    streams  = f"streams_{run_idx:03d}"
    basins   = f"basins_{run_idx:03d}"
    pert_map = f"pert_mc_{run_idx:03d}"  # created in make_perturbed_dem

    # Watershed (D8)
    gs.run_command(
        "r.watershed",
        elevation="dem_smoothed",
        accumulation=accum,
        drainage=flow_dir,
        overwrite=True
    )

    # Extract streams
    gs.run_command(
        "r.stream.extract",
        elevation="dem_smoothed",
        direction=flow_dir,
        accumulation=accum,
        threshold=STREAM_THRESHOLD,
        stream_raster=streams,
        overwrite=True
    )

    # Basins
    gs.run_command(
        "r.stream.basins",
        direction=flow_dir,
        stream_rast=streams,
        basins=basins,
        flags="l",
        overwrite=True
    )

    # -------------------------------------------------------------------------
    # Basin stability accumulator relative to reference run
    # -------------------------------------------------------------------------
    if run_idx == REF_RUN:
        # Create a dedicated reference copy
        gs.run_command("g.copy", raster=[f"{basins},basins_ref"], overwrite=True)
        print("✓ basins_ref created from reference run")

        # Initialise stability sum: this pixel matches itself in the ref run
        safe(f"{STABLE_SUM_MAP} = if(!isnull(basins_ref), 1, null())")
        print(f"✓ Initialised {STABLE_SUM_MAP} from reference basins")
    else:
        # Update stability sum: add 1 where this run's basin matches basins_ref
        expr = (
            f"{STABLE_SUM_MAP} = "
            f"if(!isnull(basins_ref) && !isnull({basins}) && {basins} == basins_ref, "
            f"{STABLE_SUM_MAP} + 1, {STABLE_SUM_MAP})"
        )
        safe(expr)
        print(f"✓ Updated {STABLE_SUM_MAP} for run {run_idx:03d}")

    # -------------------------------------------------------------------------
    # Export rasters for this realisation
    # -------------------------------------------------------------------------
    exports = [
        (basins,    fr"{OUT}\basins_mc_{run_idx:03d}.tif",       "Int32"),
        (accum,     fr"{OUT}\accum_mc_{run_idx:03d}.tif",        "Float64"),
        (flow_dir,  fr"{OUT}\flowdir_mc_{run_idx:03d}.tif",      "Int32"),
        (streams,   fr"{OUT}\streams_mc_{run_idx:03d}.tif",      "Int16"),
        (pert_map,  fr"{OUT}\perturbation_mc_{run_idx:03d}.tif", "Float32"),
    ]

    for name, fn, dtype in exports:
        fn_norm = fn.replace("\\", "/")
        try:
            info = gs.read_command("r.info", map=name)
            if "min =" in info and "max =" in info:
                print(f"📤 Exporting {name} → {fn_norm}")

                if name.startswith("pert_mc_"):
                    # For perturbation maps, be more lenient: let r.out.gdal
                    # choose type/nodata based on the GRASS raster.
                    gs.run_command(
                        "r.out.gdal",
                        input=name,
                        output=fn_norm,
                        format="GTiff",
                        createopt="COMPRESS=LZW,TILED=YES,BIGTIFF=YES",
                        overwrite=True
                    )
                else:
                    # For all the other rasters, keep explicit types/nodata
                    gs.run_command(
                        "r.out.gdal",
                        input=name,
                        output=fn_norm,
                        format="GTiff",
                        type=dtype,
                        createopt="COMPRESS=LZW,TILED=YES,BIGTIFF=YES",
                        nodata=-9999,
                        overwrite=True
                    )
            else:
                print(f"⚠️ Skipping {name}: no data detected.")
        except Exception as e:
            print(f"❌ Failed to export {name}: {e}")

    # clear mask for safety before next realisation's perturbation
    try:
        gs.run_command("r.mask", flags="r")
    except Exception:
        pass

    print(f"✅ Finished Monte Carlo hydrology for run {run_idx:03d}")


# =============================================================================
# MONTE CARLO LOOP
# =============================================================================
for i in range(1, N_MC + 1):
    print(f"\n==================== Monte Carlo Run {i:03d}/{N_MC} ====================")
    dem_mc = make_perturbed_dem(
        run_idx=i,
        base_dem="dem_filled",
        var_map="dem_var",
        corr_pix=CORR_PIX,
    )
    run_hydro_for_dem(dem_mc, i)


✅ Using existing GRASS location: C:\Users\s174035\Documents\grassdata\dem_loc
🌿 GRASS session initialized in:
   C:\Users\s174035\Documents\grassdata\dem_loc\PERMANENT

GISDBASE='C:\Users\s174035\Documents\grassdata';
LOCATION_NAME='dem_loc';
MAPSET='PERMANENT';

✓ r.in.gdal → dem
✓ r.in.gdal → dem_var
GRASS 8.4.1 (2025)

✓ Addon available: r.stream.extract
✓ Addon available: r.stream.basins
GISDBASE='C:\Users\s174035\Documents\grassdata';
LOCATION_NAME='dem_loc';
MAPSET='PERMANENT';

ADDON_BASE: C:\Users\s174035\AppData\Roaming\GRASS8\addons
PATH contains addon_bin? True
PATH contains addon_scripts? True
✓ Filled internal DEM holes → dem_filled
✓ Ice mask imported and rasterized → ice_mask_rast

==================== Monte Carlo Run 001/2 ====================
✓ Created perturbation: pert_mc_001
✓ Created perturbed DEM: dem_mc_001
✓ Mask set from ice_mask_rast
✓ basins_ref created from reference run
✓ Initialised basin_match_sum from reference basins
📤 Exporting basins_001 → E:/Rasmus/D

## 2. Pixelwise basin stability (all realisations on common grid)

This cell:

- Collects all `basins_mc_*.tif` realisations.
- Reprojects / resamples them to the grid of a chosen reference realisation.
- For each pixel, counts how often the **same basin label** appears across all runs.
- Computes a **pixelwise certainty map**:

  * `certainty(x) = n_max(x) / N`, where `n_max(x)` is the number of runs where
    the most frequent basin label occurs at pixel `x`.
  * High values (near 1) indicate very stable membership; low values highlight
    pixels near uncertain divides.

- Optionally defines a **core basin mask**, where certainty exceeds a chosen threshold.


In [2]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from pathlib import Path
from tqdm import tqdm

# =============================================================================
# USER SETTINGS
# =============================================================================
OUT_MC = Path(r"E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC")
ref_file = OUT_MC / "basins_mc_001.tif"   # reference realisation
pattern  = "basins_mc_*.tif"              # all ensemble runs
out_cert = OUT_MC / "basin_certainty_pixelwise.tif"
out_core = OUT_MC / "basins_core_pixelwise.tif"

OVERLAP_THRESH = 0.5   # min overlap_ratio to accept a match
CERT_THRESH    = 0.9   # for "core" basins (can tweak later)

# =============================================================================
# LOAD REFERENCE BASIN MAP
# =============================================================================
basin_files = sorted(f for f in OUT_MC.glob(pattern))
if ref_file not in basin_files:
    raise FileNotFoundError(f"Reference file {ref_file} not found in {OUT_MC}")

# Put reference first in the list
basin_files = [ref_file] + [f for f in basin_files if f != ref_file]
N_runs = len(basin_files)
print(f"Found {N_runs} basin realisations")

with rasterio.open(ref_file) as src_ref:
    A = src_ref.read(1)
    meta = src_ref.meta.copy()
    A_nodata = src_ref.nodata if src_ref.nodata is not None else 0
    meta.update(count=1, dtype="float32", nodata=0.0, compress="LZW")
    dst_transform, dst_crs = src_ref.transform, src_ref.crs

maskA = (A != 0) & (A != A_nodata)

# Stability counter: how many runs (including reference) consider this pixel in the "same basin"
stability = np.zeros_like(A, dtype=np.uint16)

# Reference run: by definition, every valid pixel matches itself
stability[maskA] += 1

# =============================================================================
# LOOP OVER ENSEMBLE RUNS (EXCEPT REF)
# =============================================================================
for run_idx, basin_path in enumerate(basin_files[1:], start=2):
    print(f"\n=== Comparing run {run_idx}/{N_runs}: {basin_path.name} ===")

    # --- Load and reproject B to match A's grid ---
    with rasterio.open(basin_path) as srcB:
        Bsrc = srcB.read(1)
        B_nodata = srcB.nodata if srcB.nodata is not None else 0

        B = np.full_like(A, B_nodata)
        reproject(
            source=Bsrc,
            destination=B,
            src_transform=srcB.transform,
            src_crs=srcB.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest,
            src_nodata=B_nodata,
            dst_nodata=B_nodata,
        )

    maskB = (B != 0) & (B != B_nodata)

    # For each basin ID in A, find best matching basin in B and flag overlapping pixels
    basin_ids = np.unique(A[maskA])
    basin_ids = basin_ids[basin_ids != 0]

    # local increment map for this run: 1 where pixel is considered "same basin" as ref, else 0
    same_basin_this_run = np.zeros_like(A, dtype=np.uint8)

    for bid in tqdm(basin_ids, desc="  Matching basins"):
        a_mask = (A == bid)
        area_a = np.count_nonzero(a_mask)
        if area_a == 0:
            continue

        # overlapping basins in B within this basin of A
        b_ids, b_counts = np.unique(B[a_mask & maskB], return_counts=True)
        # Remove background / nodata
        b_ids = b_ids[b_ids != 0]
        b_counts = b_counts[b_ids != 0] if len(b_ids) > 0 else b_counts

        if len(b_ids) == 0:
            # No overlap at all → we can't say this basin exists in this run
            continue

        best_b = b_ids[np.argmax(b_counts)]
        overlap_best = np.count_nonzero(a_mask & (B == best_b))
        overlap_ratio = overlap_best / area_a

        if overlap_ratio >= OVERLAP_THRESH:
            # pixels where A==bid & B==best_b are considered "same basin" as in ref
            same_basin_this_run[a_mask & (B == best_b)] = 1
        else:
            # no dominant match → basin considered unstable in this run
            pass

    # Update stability
    stability += same_basin_this_run.astype(stability.dtype)

# =============================================================================
# COMPUTE PIXEL-WISE CERTAINTY
# =============================================================================
certainty = np.zeros_like(A, dtype=np.float32)
certainty[maskA] = stability[maskA].astype(np.float32) / float(N_runs)

# =============================================================================
# BUILD "CORE" BASINS BASED ON CERTAINTY
# =============================================================================
# Start from reference basins; only keep high-certainty pixels
basins_core = np.zeros_like(A, dtype=A.dtype)
core_mask = (certainty >= CERT_THRESH) & maskA
basins_core[core_mask] = A[core_mask]

# =============================================================================
# SAVE OUTPUTS
# =============================================================================
# Certainty
with rasterio.open(out_cert, "w", **meta) as dst:
    dst.write(certainty, 1)
print("✓ Wrote pixel-wise basin certainty:", out_cert)

# Core basins
meta_core = meta.copy()
meta_core.update(dtype=A.dtype, nodata=0)
with rasterio.open(out_core, "w", **meta_core) as dst:
    dst.write(basins_core, 1)
print("✓ Wrote core basins based on certainty:", out_core)


Found 38 basin realisations

=== Comparing run 2/38: basins_mc_002.tif ===


  Matching basins: 100%|█████████████████████████████████████████████████████████████| 442/442 [00:39<00:00, 11.13it/s]



=== Comparing run 3/38: basins_mc_003.tif ===


  Matching basins: 100%|█████████████████████████████████████████████████████████████| 442/442 [00:36<00:00, 12.01it/s]



=== Comparing run 4/38: basins_mc_004.tif ===


...


=== Comparing run 37/38: basins_mc_037.tif ===


  Matching basins: 100%|█████████████████████████████████████████████████████████████| 442/442 [00:38<00:00, 11.43it/s]



=== Comparing run 38/38: basins_mc_038.tif ===


  Matching basins: 100%|█████████████████████████████████████████████████████████████| 442/442 [00:39<00:00, 11.18it/s]


✓ Wrote pixel-wise basin certainty: E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC\basin_certainty_pixelwise.tif
✓ Wrote core basins based on certainty: E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC\basins_core_pixelwise.tif


## 3. Basin‑wise stability across the ensemble

This cell looks at **whole basins** rather than individual pixels:

- For each basin in a reference realisation, finds matching basins in all other runs.
- Computes an overlap‑based stability score (e.g. fraction of runs with a good match).
- Produces maps where:

  * Stable basins (high overlap across runs) can be identified.
  * Unstable basins or those frequently split/merged across runs are highlighted.

These metrics help decide which basins are robust enough to be used for subsequent
hydrological or glaciological analysis.


In [5]:
import numpy as np
import rasterio
from pathlib import Path
from tqdm import tqdm

# =============================================================================
# USER SETTINGS
# =============================================================================
MC_DIR   = Path(r"E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC")
pattern  = "basins_mc_*.tif"      # all ensemble realisations
OVERLAP_THRESH = 0.5              # min overlap ratio to accept a basin match
CERT_THRESH    = 0.9              # for "core" basins (can tweak later)

out_cert = MC_DIR / "basin_certainty_allruns.tif"
out_core = MC_DIR / "basins_core_allruns.tif"

# =============================================================================
# COLLECT FILES
# =============================================================================
basin_files = sorted(MC_DIR.glob(pattern))
if not basin_files:
    raise FileNotFoundError(f"No files matching {pattern} in {MC_DIR}")

N_runs = len(basin_files)
print(f"Found {N_runs} basin realisations")

# Use first file as template for grid / metadata
with rasterio.open(basin_files[0]) as src0:
    template = src0.read(1)
    meta = src0.meta.copy()
    A0_nodata = src0.nodata if src0.nodata is not None else 0
    meta.update(count=1, dtype="float32", nodata=0.0, compress="LZW")

nrows, ncols = template.shape

# This will hold, for each pixel, the largest cluster size found over all references
best_cluster = np.zeros_like(template, dtype=np.uint16)

# We'll treat any non-zero, non-nodata value as "valid basin"
global_valid_mask = np.zeros_like(template, dtype=bool)

# =============================================================================
# MAIN DOUBLE LOOP: EACH RUN AS REFERENCE
# =============================================================================
for j, ref_path in enumerate(basin_files):
    print(f"\n=== Reference run {j+1}/{N_runs}: {ref_path.name} ===")

    # --- Load reference basins ---
    with rasterio.open(ref_path) as src_ref:
        A = src_ref.read(1)
        A_nodata = src_ref.nodata if src_ref.nodata is not None else 0

        # sanity check: same grid / CRS
        if src_ref.width != meta["width"] or src_ref.height != meta["height"]:
            raise ValueError(f"Grid size mismatch in {ref_path}")
        if src_ref.transform != meta["transform"]:
            raise ValueError(f"Transform mismatch in {ref_path}")
        if src_ref.crs != meta["crs"]:
            raise ValueError(f"CRS mismatch in {ref_path}")

    maskA = (A != 0) & (A != A_nodata)
    global_valid_mask |= maskA

    # cluster size anchored at this reference:
    # start with 1 where A has a basin (this run itself)
    cluster = np.zeros_like(A, dtype=np.uint16)
    cluster[maskA] = 1

    # IDs of basins in this reference
    basin_ids = np.unique(A[maskA])
    basin_ids = basin_ids[basin_ids != 0]

    # --- Loop over all OTHER runs and compare to this reference ---
    for k, other_path in enumerate(basin_files):
        if k == j:
            continue

        with rasterio.open(other_path) as srcB:
            B = srcB.read(1)
            B_nodata = srcB.nodata if srcB.nodata is not None else 0

        maskB = (B != 0) & (B != B_nodata)

        # For each basin in the reference
        for bid in basin_ids:
            a_mask = (A == bid)
            area_a = np.count_nonzero(a_mask)
            if area_a == 0:
                continue

            # What B-basins overlap this A-basin?
            vals = B[a_mask & maskB]
            if vals.size == 0:
                # this basin doesn't exist in this run at all
                continue

            b_ids, counts = np.unique(vals, return_counts=True)
            # Remove background if present
            valid_idx = (b_ids != 0)
            b_ids = b_ids[valid_idx]
            counts = counts[valid_idx]

            if len(b_ids) == 0:
                continue

            # Best-matching B basin by area overlap
            best_b = b_ids[np.argmax(counts)]
            overlap_mask = a_mask & (B == best_b)
            overlap_best = np.count_nonzero(overlap_mask)
            overlap_ratio = overlap_best / area_a

            if overlap_ratio >= OVERLAP_THRESH:
                # These pixels are "same basin" in run k as in reference j
                cluster[overlap_mask] += 1

    # Update global best cluster size
    best_cluster = np.maximum(best_cluster, cluster)
    print(f"  → max cluster size for this ref: {cluster.max()}")

# =============================================================================
# COMPUTE PIXEL-WISE CERTAINTY
# =============================================================================
certainty = np.zeros_like(template, dtype=np.float32)
certainty[global_valid_mask] = (
    best_cluster[global_valid_mask].astype(np.float32) / float(N_runs)
)

# =============================================================================
# BUILD "CORE" BASINS (OPTIONAL)
# =============================================================================
# For a core-product, we still need *some* basin labels.
# Easiest symmetric choice: take the MODE (most frequent) basin ID across all runs,
# then only keep it where certainty >= CERT_THRESH.

# Load all basins into memory (may need RAM if domain is large and N_runs big)
print("\nLoading stack of basins to compute consensus labels...")
stack = np.zeros((N_runs, nrows, ncols), dtype=template.dtype)
for i, fpath in enumerate(basin_files):
    with rasterio.open(fpath) as src:
        stack[i] = src.read(1)

consensus_basins = np.zeros_like(template, dtype=template.dtype)
core_mask = (certainty >= CERT_THRESH) & global_valid_mask

# Compute per-pixel mode label only where we care (core_mask)
print("Computing mode label for high-certainty pixels (this can take a bit)...")
rows, cols = np.where(core_mask)
for r, c in tqdm(zip(rows, cols), total=len(rows), desc="  Mode per pixel"):
    vals = stack[:, r, c]
    vals = vals[vals != 0]  # ignore background
    if vals.size == 0:
        continue
    labels, counts = np.unique(vals, return_counts=True)
    consensus_basins[r, c] = labels[np.argmax(counts)]

# =============================================================================
# SAVE OUTPUTS
# =============================================================================
# Certainty map
with rasterio.open(out_cert, "w", **meta) as dst:
    dst.write(certainty, 1)
print("✓ Wrote pixel-wise basin certainty:", out_cert)

# Core basins map
meta_core = meta.copy()
meta_core.update(dtype=template.dtype, nodata=0)
with rasterio.open(out_core, "w", **meta_core) as dst:
    dst.write(consensus_basins, 1)
print("✓ Wrote core basins based on all-run certainty:", out_core)


Found 38 basin realisations

=== Reference run 1/38: basins_mc_001.tif ===
  → max cluster size for this ref: 38

=== Reference run 2/38: basins_mc_002.tif ===


MemoryError: Unable to allocate 16.7 MiB for an array with shape (5539, 3163) and data type bool

## 4. (Optional) Extra analysis

Use this cell for plotting histograms of certainty, exploring thresholds,
or exporting summary tables for specific outlet basins of interest.
